In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, numpy as np
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
%matplotlib widget

In [ ]:
from LinearMPC.MPCVelControl import MPCVelControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis, plot_static_states_inputs

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:
Ts = 0.05
sim_time = 15.0 
H = 5.0
x0 = np.array([0, 0, 0, 0, 0, 0, 5, 5, 10, 0, 0, 1])
x_target = np.zeros((12,))

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl.new_controller(rocket, Ts, H)

rocket.mass = 2.0
rocket.fuel_rate = 0.1

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, ref = rocket.simulate_control(
    mpc, sim_time, H, x0, x_target=x_target, method='nonlinear'
)

print(f"\nSimulation completed. Final time: {t_cl[-1]:.2f}s")
print(f"Final position: x={x_cl[9,-1]:.2f}, y={x_cl[10,-1]:.2f}, z={x_cl[11,-1]:.2f}")
print(f"Final velocity: vx={x_cl[6,-1]:.2f}, vy={x_cl[7,-1]:.2f}, vz={x_cl[8,-1]:.2f}")

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, Ref=ref[:, :-1], T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol);

In [ ]:
# Plot the trajectory
plot_static_states_inputs(t_cl[:-1], x_cl[:, :-1], u_cl, ref[:, :-1])


## Detailed Analysis: Time-Varying Mass Effects

Analyze how fuel consumption affects mass, disturbance estimation, and tracking performance.

In [ ]:
import matplotlib.pyplot as plt

# Re-run simulation with data collection for analysis
print("Running simulation with detailed data collection...")

rocket_analysis = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc_analysis = MPCVelControl.new_controller(rocket_analysis, Ts, H)
rocket_analysis.mass = 2.0
rocket_analysis.fuel_rate = 0.1

# Storage for analysis
mass_history = []
fuel_consumption_history = []
d_hat_history = []
x_hat_history = []
vz_error_history = []
altitude_history = []

x_current = x0.copy()
for i in range(len(t_cl) - 1):  
    # Store current effective mass
    mass_history.append(rocket_analysis.mass - rocket_analysis.fuel_consumption)
    fuel_consumption_history.append(rocket_analysis.fuel_consumption)

    # Get control input
    u, _, _, _ = mpc_analysis.get_u(0, x_current, x_target=x_target)

    # Store disturbance estimate and tracking error
    d_hat_history.append(mpc_analysis.mpc_zvel.d_hat[0])
    x_hat_history.append(mpc_analysis.mpc_zvel.x_hat[0])
    vz_error_history.append(x_current[8])  # vz error 
    altitude_history.append(x_current[11])  # z position

    # Simulate one step
    x_current = rocket_analysis.simulate_step(x_current, Ts, u, method='nonlinear')

# Add final values
mass_history.append(rocket_analysis.mass - rocket_analysis.fuel_consumption)
fuel_consumption_history.append(rocket_analysis.fuel_consumption)
d_hat_history.append(mpc_analysis.mpc_zvel.d_hat[0])
x_hat_history.append(mpc_analysis.mpc_zvel.x_hat[0])
vz_error_history.append(x_current[8])
altitude_history.append(x_current[11])

mass_history = np.array(mass_history)
fuel_consumption_history = np.array(fuel_consumption_history)
d_hat_history = np.array(d_hat_history)
x_hat_history = np.array(x_hat_history)
vz_error_history = np.array(vz_error_history)
altitude_history = np.array(altitude_history)
t_analysis = t_cl[:len(d_hat_history)]

print(f"\nMass Analysis:")
print(f"  Initial mass: {mass_history[0]:.3f} kg")
print(f"  Final mass: {mass_history[-1]:.3f} kg")
print(f"  Mass consumed: {mass_history[0] - mass_history[-1]:.3f} kg")
print(f"  Final disturbance estimate: d_hat = {d_hat_history[-1]:.6f}")

# Check if simulation ended early
fuel_exhausted = False
if len(t_cl) < int(sim_time / Ts):
    print(f"\n  ⚠️ Simulation ended early (fuel exhausted at t={t_cl[-1]:.2f}s)")
    fuel_exhausted = True

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(14, 11))
fig.suptitle('Time-Varying Mass Analysis\n' +
             f'Initial mass=2.0 kg, fuel_rate=0.1, Final mass={mass_history[-1]:.2f} kg',
             fontsize=14, fontweight='bold')

# Plot 1: Z velocity tracking
ax = axes[0, 0]
ax.plot(t_cl, x_cl[8, :], 'b-', linewidth=2, label='vz (actual)')
ax.axhline(0, color='r', linestyle='--', linewidth=1, alpha=0.7, label='vz reference')
ax.set_xlabel('Time [s]')
ax.set_ylabel('vz [m/s]')
ax.set_title('Z-Velocity Tracking')
ax.legend()
ax.grid(True, alpha=0.3)
if fuel_exhausted:
    ax.axvline(t_cl[-1], color='k', linestyle=':', linewidth=1.5, alpha=0.5, label='Fuel exhausted')

# Plot 2: Altitude (z position)
ax = axes[0, 1]
ax.plot(t_cl, x_cl[11, :], 'g-', linewidth=2)
ax.axhline(0, color='r', linestyle='--', linewidth=1, alpha=0.5, label='Ground')
ax.set_xlabel('Time [s]')
ax.set_ylabel('z [m]')
ax.set_title('Altitude')
ax.legend()
ax.grid(True, alpha=0.3)
if fuel_exhausted:
    ax.axvline(t_cl[-1], color='k', linestyle=':', linewidth=1.5, alpha=0.5)

# Plot 3: Mass over time
ax = axes[1, 0]
ax.plot(t_analysis, mass_history, 'purple', linewidth=2, label='Rocket mass')
ax.axhline(1.0, color='r', linestyle='--', linewidth=1, alpha=0.5, label='Empty mass (no fuel)')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Mass [kg]')
ax.set_title('Rocket Mass (Fuel Consumption)')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Disturbance estimate d_hat
ax = axes[1, 1]
ax.plot(t_analysis, d_hat_history, 'orange', linewidth=2, label='d_hat (estimated)')
ax.set_xlabel('Time [s]')
ax.set_ylabel('d (disturbance)')
ax.set_title('Disturbance Estimate')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 5: Control input (Pavg)
ax = axes[2, 0]
ax.plot(t_cl[:-1], u_cl[2, :], 'b-', linewidth=2, label='Pavg')
ax.axhline(40, color='r', linestyle='--', linewidth=1, alpha=0.5)
ax.axhline(80, color='r', linestyle='--', linewidth=1, alpha=0.5)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Pavg [%]')
ax.set_title('Z-Control Input')
ax.legend()
ax.grid(True, alpha=0.3)
if fuel_exhausted:
    ax.axvline(t_cl[-1], color='k', linestyle=':', linewidth=1.5, alpha=0.5)

# Plot 6: Tracking error over time
ax = axes[2, 1]
ax.plot(t_analysis, vz_error_history, 'r-', linewidth=2, label='vz tracking error')
ax.axhline(0, color='k', linestyle='--', linewidth=1, alpha=0.5)
ax.set_xlabel('Time [s]')
ax.set_ylabel('vz error [m/s]')
ax.set_title('Z-Velocity Tracking Error')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
#with the help of copilot